In [472]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

In [473]:
# Constants

gammac = 1.4
gammah = 1.33

cpa = 1.005
cp = 1.148
R = 287

In [474]:
# Compressor Values

Ca4 = 231.16 # m/s
Ca5 = Ca4
N = 208.05 # Rev/s
deltaTocomp = 439.58 # K

In [475]:
To4 = 2275 # K
polyt = 0.92
#flowcoeff Greater than 0.78
#loadcoeff Cannot Exceed 3.3
mfahot = 40 # kg/s
mfthot = mfahot + (mfahot*.02)

In [476]:
# Turbine PARAMETERS

htratioturbine = 0.715

In [477]:
deltaToturb = (deltaTocomp*cpa*0.99)/cp

T4 = To4-((Ca4**2)/(2*cp*1000))

po4 = 15 # bar
p4 = po4*(T4/To4)**(gammah/(gammah-1))

density4 = (p4*100000)/(R*T4)

print(density4)

2.2268738031901054


In [478]:
A4 = mfthot/(density4*Ca4)

M4 = Ca4/(np.sqrt(1.333*R*To4))

print("Turbine Inlet Area = %3.4f m^2" % (A4))

print("Turbine Inlet Mach Number = %3.2f" % (M4))

Turbine Inlet Area = 0.0793 m^2
Turbine Inlet Mach Number = 0.25


In [479]:
r4t = np.sqrt(mfthot/(np.pi*density4*Ca4*(1-(htratioturbine**2))))

r4h = htratioturbine*r4t

r4m = (r4t+r4h)/2

print("r4t = %3.3f m" % (r4t))
print("rm = %3.3f m" % (r4m))
print("r4h = %3.3f m" % (r4h))

r4t = 0.227 m
rm = 0.195 m
r4h = 0.162 m


In [480]:
To5 = To4 - deltaToturb

T5 = To5 - (Ca5**2)/(2*cp*1000)

po5 = po4*((To5/To4)**((gammah*polyt)/(gammah-1)))

p5 = po5*(T5/To5)**(gammah/(gammah-1))

density5 = (p5*100000)/(R*T5)

A5 = mfthot/(density5*Ca5)

print("Turbine Exhaust Area = %3.4f m^2" % (A5))

Turbine Exhaust Area = 0.1310 m^2


In [481]:
# constant mean radius
#bladeheight5 = A5/(2*np.pi*rm)
#
#r5t = rm+(bladeheight5/2)
#
#r5h = rm-(bladeheight5/2)


# constant hub radius
r5h = r4h
r5t = np.sqrt(A5/np.pi + (r5h)**2)
r5m = (r5t+r5h)/2

print("r5t = %3.3f m" % (r5t))
print("r5m = %3.3f m" % (r5m))
print("r5h = %3.3f m" % (r5h))

r5t = 0.261 m
r5m = 0.212 m
r5h = 0.162 m


In [482]:
U5h = r5h*(2*np.pi*N)

print(U5h)

flow5 = Ca5/U5h

print(flow5)

bladeloading = 3.3

deltaTos = (bladeloading*(U5h**2))/(2*cp*1000)

print(deltaTos)

stageest = deltaToturb/deltaTos

print("Estimation of %3.1f Stages" % (stageest))

212.3489594716087
1.0885855083782803
64.81004614225924
Estimation of 5.9 Stages


In [483]:
U4t = 2*np.pi*r4t*N
U4m = 2*np.pi*r4m*N
U4h = 2*np.pi*r4h*N

In [484]:
stageactual = 6
# Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff
stagevals = np.zeros((stageactual+1,3,17))
stagevals[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca4,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca4,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca4,0,0,0,0,0,po4,0,0]])

In [485]:
def initialvals4(deltaTocomp,Ca4,To4,po4,htratioturbine):
    deltaToturb = (deltaTocomp*cpa*0.99)/cp

    T4 = To4-((Ca4**2)/(2*cp*1000))

    po4 = 15 # bar
    p4 = po4*(T4/To4)**(gammah/(gammah-1))

    density4 = (p4*100000)/(R*T4)

    A4 = mfthot/(density4*Ca4)

    M4 = Ca4/(np.sqrt(1.333*R*To4))

    r4t = np.sqrt(mfthot/(np.pi*density4*Ca4*(1-(htratioturbine**2))))

    r4h = htratioturbine*r4t

    r4m = (r4t+r4h)/2

    return deltaToturb,r4t,r4h,r4m

In [486]:
def initialvals5(deltaToturb,Ca5,To4,po4,r4h):
    To5 = To4 - deltaToturb

    T5 = To5 - (Ca5**2)/(2*cp*1000)

    po5 = po4*((To5/To4)**((gammah*polyt)/(gammah-1)))

    p5 = po5*(T5/To5)**(gammah/(gammah-1))

    density5 = (p5*100000)/(R*T5)

    A5 = mfthot/(density5*Ca5)

    r5h = r4h

    r5t = np.sqrt(A5/np.pi+(r5h)**2)

    r5m = (r5t+r5h)/2

    return r5t,r5h,r5m

In [487]:
def estimatevals(r5h,N,Ca5,bladeloading,deltaToturb):
    U5h = r5h*(2*np.pi*N)

    flow5 = Ca5/U5h

    deltaTos = (bladeloading*(U5h**2))/(2*cp*1000)

    stageest = deltaToturb/deltaTos

    return stageest

$$
\psi = \frac{2 c_p \Delta T_{0s}}{U^2} \\
\phi = \frac{C_a}{U} \\
tan \beta_2 = \frac{1}{2 \phi} (\frac{\psi}{2} - 2\Lambda) \\
tan \beta_3 = \frac{1}{2 \phi} (\frac{\psi}{2} + 2\Lambda) \\
$$

In [488]:
def returnstage(deltaTo,U,Ca,DOR):
    bladeloading = (2*cp*1000*deltaTo)/(U**2)
    flow = Ca/U
    beta2 = np.degrees(np.arctan((1/(2*flow))*((bladeloading/2)-(2*DOR))))
    beta3 = np.degrees(np.arctan((1/(2*flow))*((bladeloading/2)+(2*DOR))))
    alpha2 = np.degrees(np.arctan(np.tan(np.radians(beta2))+(1/flow)))
    alpha3 = np.degrees(np.arctan((np.tan(np.radians(beta3))-(1/flow))))

    Cw2 = Ca*np.tan(np.radians(alpha2))
    Cw3 = Ca*np.tan(np.radians(alpha3))

    V2 = Ca/(np.cos(np.radians(beta2)))
    V3 = Ca/(np.cos(np.radians(beta3)))

    return beta2,beta3,alpha2,alpha3,Cw2,Cw3,V2,V3,bladeloading,flow

In [489]:
def iterate(stagevals,stageactual,deltaTosact,r4t,r4m,r4h,N,Ca):
    i = 1

    # This runs through the params for each stage and saves them to an array
    while i < stageactual+1:
        if i == 0:
            DOR = 0.5
        if i == 1:
            DOR = 0.55
        if i == 2 or i == 3:
            DOR = 0.6
        if i == 4 or i == 5:
            DOR = 0.7
        else:
            DOR = 0.8

        rtstage = ((r5t-r4t)/(stageactual-1))*i+r4t 
        rmstage = ((r5m-r4m)/(stageactual-1))*i+r4m
        rhstage = ((r5h-r4h)/(stageactual-1))*i+r4h

        Utstage = 2*np.pi*rtstage*N
        Umstage = 2*np.pi*rmstage*N
        Uhstage = 2*np.pi*rhstage*N

        stageTo3 = stagevals[i-1,0,6] - deltaTosact

        pr = ((stageTo3/stagevals[i-1,0,6])**((gammah*polyt)/(gammah-1)))

        # DO NOT TOUCH THE COMMENTED FUNCTION UNLESS THE FIRST STAGE VALUES ARE FUCKED UP

        """if i == 1:
            deltawhirlt = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Utstage))
            Cw2t = deltawhirlt - Cw1t
            stagebeta1tip = np.degrees(np.arctan(Utstage/Castage))
            stagebeta2tip = np.degrees(np.arctan((Utstage-Cw2t)/Castage))
            stagealpha1tip = 0
            stagealpha2tip = np.degrees(np.arctan((Cw2t)/Castage))

            V1t = Castage/(np.cos(np.radians(stagebeta1tip)))
            dehallert = np.cos(np.radians(stagebeta1tip))/np.cos(np.radians(stagebeta2tip))
            V2t = V1t*dehallert

            deltawhirlm = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Umcomp))
            Cw2m = deltawhirlm - Cw1m
            stagebeta1mean = np.degrees(np.arctan(Umcomp/Castage))
            stagebeta2mean = np.degrees(np.arctan((Umcomp-Cw2m)/Castage))
            stagealpha1mean = 0
            stagealpha2mean = np.degrees(np.arctan((Cw2m)/Castage))

            V1m = Castage/(np.cos(np.radians(stagebeta1mean)))
            dehallerm = np.cos(np.radians(stagebeta1mean))/np.cos(np.radians(stagebeta2mean))
            V2m = V1m*dehallerm

            deltawhirlh = ((1.005*(10**3)*(stagedeltaTo))/(lambcomp*Uhcomp))
            Cw2h = deltawhirlh - Cw1h
            stagebeta1hub = np.degrees(np.arctan(Uhcomp/Castage))
            stagebeta2hub = np.degrees(np.arctan((Uhcomp-Cw2h)/Castage))
            stagealpha1hub = 0
            stagealpha2hub = np.degrees(np.arctan((Cw2h)/Castage))

            V1h = Castage/(np.cos(np.radians(stagebeta1hub)))
            dehallerh = np.cos(np.radians(stagebeta1hub))/np.cos(np.radians(stagebeta2hub))
            V2h = V1h*dehallerh
        else: """
        stagebeta2tip,stagebeta3tip,stagealpha2tip,stagealpha3tip,Cw2t,Cw3t,V2t,V3t,bladelt,flowt = returnstage(deltaTosact,Utstage,Ca,DOR)

        stagebeta2mean,stagebeta3mean,stagealpha2mean,stagealpha3mean,Cw2m,Cw3m,V2m,V3m,bladelm,flowm = returnstage(deltaTosact,Umstage,Ca,DOR)

        stagebeta2hub,stagebeta3hub,stagealpha2hub,stagealpha3hub,Cw2h,Cw3h,V2h,V3h,bladelh,flowh = returnstage(deltaTosact,Uhstage,Ca,DOR)

        #pr = (1+((polyc*(((Umstage*Castage)/(cpa*1000))*(np.tan(np.radians(stagebeta1mean))-np.tan(np.radians(stagebeta2mean)))))/(stagevals[i-1,0,6])))**(gammac/(gammac-1))
        stagepo3 = stagevals[i-1,0,14]*pr

        stagevals[i,:,:] = np.array([[stagealpha2hub,stagealpha3hub,stagebeta2hub,stagebeta3hub,rhstage,deltaTosact,stageTo3,Uhstage,Ca,Cw2h,Cw3h,V2h,V3h,pr,stagepo3,bladelh,flowh],[stagealpha2mean,stagealpha3mean,stagebeta2mean,stagebeta3mean,rmstage,deltaTosact,stageTo3,Umstage,Ca,Cw2m,Cw3m,V2m,V3m,pr,stagepo3,bladelm,flowm],[stagealpha2tip,stagealpha3tip,stagebeta2tip,stagebeta3tip,rtstage,deltaTosact,stageTo3,Utstage,Ca,Cw2t,Cw3t,V2t,V3t,pr,stagepo3,bladelt,flowt]])

        i += 1

    return stagevals

In [490]:
deltaTosact = deltaToturb/stageactual
print(deltaTosact)
stagevals = iterate(stagevals,stageactual,deltaTosact,r4t,r4m,r4h,N,Ca4)

63.495952526132406


In [491]:
def valtable(location,vals):
    if location == 0:
        print("Hub Values:")
    elif location == 1:
        print("Mean Values:")
    elif location == 2:
        print("Tip Values:")
    
    num = 0
    stages = ["Inlet"]

    while num < len(vals[:,location,4])-1:
        stages.append("Stage "+str(num+1))
        num += 1

    pd.set_option("display.precision", 4)
    tabletry = pd.DataFrame({"Alpha 2": vals[:,location,0],"Alpha 3": vals[:,location,1],"Beta 2": vals[:,location,2],"Beta 3": vals[:,location,3],"Radius": vals[:,location,4],"Delta To": vals[:,location,5],"To2": vals[:,location,6],"U": vals[:,location,7],"Ca": vals[:,location,8],"Cw1": vals[:,location,9],"Cw2": vals[:,location,10],"V1": vals[:,location,11],"V2": vals[:,location,12],"PR": vals[:,location,13],"po2": vals[:,location,14],"Blade Loading": vals[:,location,15],"Flow": vals[:,location,16]},index=stages)

    return tabletry

In [492]:
valtable(0,stagevals)

Hub Values:


,Alpha 2,Alpha 3,Beta 2,Beta 3,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,2275.0000,212.349,231.16,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,42.8065,29.1953,0.4354,55.9072,0.1624,63.496,2211.5040,212.349,231.16,214.1056,129.1660,231.1667,412.3923,0.9004,13.5054,3.2331,1.0886
Stage 2,42.8065,29.1953,0.4354,55.9072,0.1624,63.496,2148.0081,212.349,231.16,214.1056,129.1660,231.1667,412.3923,0.8976,12.1226,3.2331,1.0886
Stage 3,42.8065,29.1953,0.4354,55.9072,0.1624,63.496,2084.5121,212.349,231.16,214.1056,129.1660,231.1667,412.3923,0.8947,10.8462,3.2331,1.0886
Stage 4,45.5134,25.0284,5.6800,54.1804,0.1624,63.496,2021.0162,212.349,231.16,235.3405,107.9311,232.3006,394.9864,0.8916,9.6708,3.2331,1.0886
Stage 5,45.5134,25.0284,5.6800,54.1804,0.1624,63.496,1957.5202,212.349,231.16,235.3405,107.9311,232.3006,394.9864,0.8884,8.5913,3.2331,1.0886
Stage 6,42.8065,29.1953,0.4354,55.9072,0.1624,63.496,1894.0243,212.349,231.16,214.1056,129.1660,231.1667,412.3923,0.8849,7.6026,3.2331,1.0886


In [493]:
valtable(1,stagevals)

Mean Values:


,Alpha 2,Alpha 3,Beta 2,Beta 3,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,2275.0000,254.6703,231.16,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,39.7850,21.0271,-16.0699,56.4014,0.1982,63.496,2211.5040,259.0822,231.16,192.4926,88.8597,240.5600,417.7302,0.9004,13.5054,2.1719,0.8922
Stage 2,39.5687,20.3247,-17.4075,56.4903,0.2016,63.496,2148.0081,263.4940,231.16,191.0195,85.6219,242.2550,418.7086,0.8976,12.1226,2.0998,0.8773
Stage 3,39.3625,19.6328,-18.7086,56.5846,0.2049,63.496,2084.5121,267.9059,231.16,189.6240,82.4616,244.0553,419.7528,0.8947,10.8462,2.0312,0.8628
Stage 4,42.9965,12.7116,-13.8013,54.5323,0.2083,63.496,2021.0162,272.3178,231.16,215.5341,52.1434,238.0322,398.3843,0.8916,9.6708,1.9659,0.8489
Stage 5,42.8889,11.8936,-15.0154,54.6118,0.2117,63.496,1957.5202,276.7297,231.16,214.7239,48.6860,239.3317,399.1621,0.8884,8.5913,1.9037,0.8353
Stage 6,38.8014,17.6184,-22.3996,56.8965,0.2151,63.496,1894.0243,281.1416,231.16,185.8665,73.4098,250.0246,423.2513,0.8849,7.6026,1.8445,0.8222


In [494]:
valtable(2,stagevals)

Tip Values:


,Alpha 2,Alpha 3,Beta 2,Beta 3,Radius,Delta To,To2,U,Ca,Cw1,Cw2,V1,V2,PR,po2,Blade Loading,Flow
Inlet,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,2275.0000,296.9916,231.16,0.0000,0.0000,0.0000,0.0000,0.0000,15.0000,0.0000,0.0000
Stage 1,37.9599,14.0889,-28.4931,57.5702,0.2339,63.496,2211.5040,305.8153,231.16,180.3418,58.0156,263.0182,431.0544,0.9004,13.5054,1.5588,0.7559
Stage 2,37.7161,12.8920,-30.4468,57.8332,0.2407,63.496,2148.0081,314.6391,231.16,178.7643,52.9086,268.1360,434.1961,0.8976,12.1226,1.4726,0.7347
Stage 3,37.4990,11.7269,-32.2930,58.1050,0.2474,63.496,2084.5121,323.4629,231.16,177.3691,47.9839,273.4563,437.5017,0.8947,10.8462,1.3934,0.7146
Stage 4,42.1683,2.4767,-28.0012,55.9671,0.2542,63.496,2021.0162,332.2867,231.16,209.3704,9.9984,261.8079,413.0303,0.8916,9.6708,1.3204,0.6957
Stage 5,42.1424,1.1187,-29.7148,56.2247,0.2609,63.496,1957.5202,341.1105,231.16,209.1803,4.5140,266.1588,415.8019,0.8884,8.5913,1.2529,0.6777
Stage 6,36.9918,8.4076,-37.2526,58.9596,0.2677,63.496,1894.0243,349.9343,231.16,174.1398,34.1661,290.4112,448.2946,0.8849,7.6026,1.1905,0.6606


In [495]:
#check = False

In [496]:
"""end = 0
Ca = 150
while not check:
    HTratio = 0.3
    while HTratio < .8:
        N = 220
        while N < 250:
            deltaToturb,r4t,r4h,r4m = initialvals4(deltaTocomp,Ca,To4,po4,HTratio)
            r5t,r5h,r5m = initialvals5(deltaToturb,Ca,To4,po4,r4h)
            stageest = estimatevals(r5h,N,Ca,3.3,deltaToturb)
            stageactuala = np.ceil(stageest).astype('int32')
            # Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff
            stagevalsa = np.zeros((stageactuala+1,3,17))
            stagevalsa[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca,0,0,0,0,0,po4,0,0]])
            deltaTosact = deltaToturb/stageactuala
            stagevalsa = iterate(stagevalsa,stageactuala,deltaTosact,r4t,r4m,r4h,N,Ca)
            
            if stagevalsa[1,0,15] < 3.3 and stagevalsa[stageactuala,2,16] > 0.78 and stagevalsa[1,0,16] > 0.78 and stagevalsa[stageactuala,2,7] < 290:
                print("Stagecount:"+str(stageactuala))
                print(len(stagevalsa[:,0,4]))
                print(stagevalsa)
                check = True
            N += 1
        HTratio += .1

    Ca += 0.1
    if end > 10000:
        check = True
        print("BAH")
    end += 1

print(HTratio)
print(N)
print(Ca)
print(end)"""

'end = 0\nCa = 150\nwhile not check:\n    HTratio = 0.3\n    while HTratio < .8:\n        N = 220\n        while N < 250:\n            deltaToturb,r4t,r4h,r4m = initialvals4(deltaTocomp,Ca,To4,po4,HTratio)\n            r5t,r5h,r5m = initialvals5(deltaToturb,Ca,To4,po4,r4h)\n            stageest = estimatevals(r5h,N,Ca,3.3,deltaToturb)\n            stageactuala = np.ceil(stageest).astype(\'int32\')\n            # Values per stage: alpha2, alpha3, beta2, beta3, r, deltaT, T3, U, Cw2, Cw3, V2, V3, pr, po3, bladeloading, flowcoeff\n            stagevalsa = np.zeros((stageactuala+1,3,17))\n            stagevalsa[0,:,:] = np.array([[0,0,0,0,0,0,To4,U4h,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4m,Ca,0,0,0,0,0,po4,0,0],[0,0,0,0,0,0,To4,U4t,Ca,0,0,0,0,0,po4,0,0]])\n            deltaTosact = deltaToturb/stageactuala\n            stagevalsa = iterate(stagevalsa,stageactuala,deltaTosact,r4t,r4m,r4h,N,Ca)\n            \n            if stagevalsa[1,0,15] < 3.3 and stagevalsa[stageactuala,2,16] > 0.78